In [1]:
import datetime
import os
from datetime import date, timedelta
from time import strptime
from urllib.parse import urlencode

import requests as q

import sys
from datetime import datetime

import click

today = date.today()
start_date = today - timedelta(days=1)

class QUANDLDATA(object):
    data = []
    currency_pair = "FB"
    source = "WIKI"
    end_date = today.strftime("%Y-%m-%d")
    start_date = start_date.strftime("%Y-%m-%d")
    url = 'https://www.quandl.com/api/v3/datasets/{0}/{1}/{2}?'
    api_key = os.environ.get("QUANDL_API_KEY")
    return_format = 'data.json'

    def __init__(self, *args, **kwargs):
        if kwargs.pop('currency_pair', None) is not None:
            self.currency_pair = kwargs.get('currency_pair')
        if kwargs.pop('source', None) is not None:
            self.source = kwargs.get('source')

    def date_range(self, start_date, end_date):
        if start_date is not None:
            self.start_date = start_date.strftime("%Y-%m-%d")
        if end_date is not None:
            self.end_date = end_date.strftime("%Y-%m-%d")
        return self.start_date, self.end_date

    def get_data(self, start_date=None, end_date=None,
                 currency_pair=None, source=None, **kwargs):
        """Get sample data

        Args:
            start_date (date, optional): start date 
            end_date (date, optional): end date

        Returns:
            data
        """
        self.currency_pair = currency_pair if currency_pair else self.currency_pair  # noqa
        self.source = source if source else self.source
        start, end = self.date_range(start_date, end_date)

        url = self.url_parse(start_date=self.start_date,
                             end_date=self.end_date, **kwargs)
        
        data = q.get(url, {'API_KEY': self.api_key})
        if data:
            self.data = data.json()
            self.has_been_called = True
            return self.data
        raise Exception(data.json())

    def get_metadata(self):
        assert self.has_been_called == True, "call get data"  # noqa
        meta = self.data.get("dataset_data")
        return meta

    def url_parse(self, **kwargs):
        params = urlencode(kwargs)
        url = self.url.format(
            self.source, self.currency_pair, self.return_format) + params
        return url


In [ ]:
CURRENCY_OPTIONS = ("EURAUD", "EURBGN",
                    "EURCAD", "EURCHF",
                    "EURCNY", "EURGBP",
                    "EURINR", "EURNZD",
                    "EURUSD", "XUDLADS", "XUDLSKS"
                    )
SOURCES = ("BOE", "ECB", "WIKI", "FRED")

In [2]:
import pandas as pd

import plotly as plt
import plotly.graph_objs as go
from plotly import tools

class Plot(object):

    def __init__(self, *args, **kwargs):
        for key, value in kwargs.items():
            if value is not None:
                setattr(self, key, value)

    def __clean_data(self, request):
        data = request.get("dataset_data").get("data")
        headers = request.get("dataset_data").get("column_names")
        clean_data = pd.DataFrame(data, None, list(
            map(lambda x: x.lower(), headers)))
        clean_data.date = pd.to_datetime(clean_data.date, format="%Y-%M-%d")
        clean_data.set_index(clean_data.date, inplace=True)
        return clean_data

    def plot(self):
        q = QUANDLDATA()
        self.data = q.get_data(**self.__dict__, collapse='daily')
        clean_data = self.__clean_data(self.data)
        ma = clean_data.close.rolling(center=False, window=30).mean()

        trace = go.Ohlc(x=clean_data.date, open=clean_data.open,
                        close=clean_data.close, high=clean_data.high,
                        low=clean_data.low, name="Currencies")
        trace1 = go.Scatter(x=clean_data.index, y=ma, name="Moving average")
        trace2 = go.Bar(x=clean_data.index, y=clean_data.volume, name="Volume")

        data = [trace, trace1, trace2]

        fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True)
        fig.append_trace(trace, 1, 1)
        fig.append_trace(trace1, 1, 1)
        fig.append_trace(trace2, 2, 1)
        plt.offline.plot(fig, filename="text.html")
        return plt


In [3]:
import sys
from datetime import datetime

import click

def plot(*args, **kwargs):
    graph = Plot(**kwargs)
    graph.plot()
    
plot()

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

